In [1]:
import pennylane as qml, numpy as np

def CliffordRX(angle: int, wires: "qml.Wires"):
    """
    Returns the exp(-i theta/2 X) operator,
    for theta ∈ {0, pi/2, pi, 3pi/2},
    decomposed into basis operators accepted by the
    qml 'default.clifford' device.

    Args:
        integer_angle (int): theta / (pi/2)
    """
    if not isinstance(angle, (int, np.integer)):
        print(type(angle))
        raise TypeError(f"angle = {angle} should be integer")
    if angle % 4 == 0:
        #qml.I(wires)
        return
    elif angle % 4 == 1:
        qml.adjoint(qml.S(wires))
        qml.H(wires)
        qml.adjoint(qml.S(wires))
    elif angle % 4 == 2:
        qml.X(wires)
    elif angle % 4 == 3:
        qml.S(wires)
        qml.H(wires)
        qml.S(wires)

def CliffordRY(angle: int, wires: "qml.Wires"):
    if not isinstance(angle, (int, np.integer)):
        print(type(angle))
        raise TypeError(f"angle = {angle} is of type {type(angle)}. It should be integer")
    if angle % 4 == 0:
        #qml.I(wires)
        return
    elif angle % 4 == 1:
        qml.H(wires)
        qml.X(wires)
    elif angle % 4 == 2:
        qml.Y(wires)
    elif angle % 4 ==3:
        qml.X(wires)
        qml.H(wires)

def random_angles(size: int | list[int]) -> np.ndarray:
    possible_thetas = np.arange(4)
    return np.random.choice(possible_thetas, size)

def unit_vector(index: int, size: int):
    return np.array([
        1 if _ == index else 0
        for _ in range(size)
    ])

def basis_encoding(x: int, wires: "qml.Wires"):
    xbin = np.binary_repr(x, wires.__len__())
    for i in range(len(wires)):
        if xbin[i] == '1':
            qml.X(wires[i])

def simple_test_circuit(angles: list[int], wires: "qml.Wires"):
    n_qubits = len(angles)
    for i in range(n_qubits):
        CliffordRX(angles[i], wires[i])

def sum_of_expvals(f):
    """
    Decorator to make quantum functions return a sum of measured operators rather than
    an array of them.
    """
    def wrapper(*args, **kwargs):
        return np.sum(f(*args, **kwargs))
    return wrapper

def parameter_shift_rule_gradient(f: "function", angles: np.ndarray[int], *args, **kwargs):
    out = np.empty_like(angles)
    for i in np.ndindex(angles.shape):
        original_angle = angles[i]  # Will need to be restored at the end of loop
        angles[i] += 1
        f_plus = f(angles, *args, **kwargs)
        angles[i] -= 2
        f_minus = f(angles, *args, **kwargs)
        angles[i] = original_angle
        out[i] = (f_plus - f_minus) / 2
    return out

def empirical_NTK(f: "function", angles: np.ndarray[int], x, y):
    return np.sum(
        parameter_shift_rule_gradient(f, angles, x)
        *
        parameter_shift_rule_gradient(f, angles, y)
    )

def analytic_NTK(f: "function", x, y, angles_shape: tuple[int], n_shots: int):
    return np.mean([
        empirical_NTK(
            f, random_angles(angles_shape), x, y
        )
        for _ in range(n_shots)
    ])

def elementary_2_qubit_gate(angles: np.ndarray[int], wires: "qml.Wires"):
    if np.shape(angles) != (4,):
        raise ValueError(f"angles has shape {np.shape(angles)}. It should be (4,)")
    if len(wires) != 2:
        raise ValueError(f"This gate acts on 2 qubits, but {len(wires)} wires were provided.")
    #print(wires)
    CliffordRX(angles[0], wires[0])
    CliffordRX(angles[1], wires[1])
    qml.CNOT([wires[0], wires[1]])
    CliffordRY(angles[2], wires[0])
    CliffordRY(angles[3], wires[1])


def layer(angles: np.ndarray[int], wires: "qml.Wires"):
    if np.shape(angles) != (4, 21):
        raise ValueError(f"angles has shape {np.shape(angles)}. It should be (4,21)")
    if len(wires) != 10:
        raise ValueError(f"This gate acts on 10 qubits, but {len(wires)} wires were provided.")
    
    i = 0
    def eg(wires):
        nonlocal i
        elementary_2_qubit_gate(angles[:, i], wires)
        i+=1
    
    eg([wires[8], wires[9]]); eg([wires[6], wires[7]])
    eg([wires[7], wires[9]]); eg([wires[6], wires[8]])
    eg([wires[8], wires[9]]); eg([wires[6], wires[7]]); eg([wires[4], wires[5]])
    eg([wires[5], wires[7]]); eg([wires[4], wires[6]])
    eg([wires[6], wires[7]]); eg([wires[4], wires[5]]); eg([wires[2], wires[3]])
    eg([wires[3], wires[5]]); eg([wires[2], wires[4]])
    eg([wires[4], wires[5]]); eg([wires[2], wires[3]]); eg([wires[0], wires[1]])
    eg([wires[1], wires[3]]); eg([wires[0], wires[2]])
    eg([wires[2], wires[3]]); eg([wires[0], wires[1]])

def paolo_circuit(angles: np.ndarray[int], wires):
    if np.shape(angles) != (4, 21, 3):
        raise ValueError(f"angles has shape {np.shape(angles)}. It should be (4,21,3)")
    for i in range(3):
        #print(f"Building layer {i}")
        layer(angles[:,:,i], wires)
        qml.I(wires[0])




In [35]:
wires = [f"{letter}{i}" for i in range(5) for letter in ["y", "x"]]
dev = qml.device('default.qubit', wires = wires)

#@sum_of_expvals
@qml.qnode(dev)
def paolo_f(angles):
    qml.AmplitudeEmbedding([1/np.sqrt(2), 1/np.sqrt(2)], dev.wires, 0, True)
    qml.Snapshot("Encoded state")
    qml.Snapshot(measurement=qml.expval(qml.Z("x4")))
    paolo_circuit(angles, dev.wires)
    #print("paolo_circuit succesfully completed")
    ops = [qml.expval(qml.Z(wire)) for wire in dev.wires]
    #print("ops successfully computed")
    return ops

In [36]:
angles = np.zeros([4, 21, 3], dtype=int)
#angles[2,18,2] = 2
print(qml.draw(paolo_f)(angles))

y0: ─╭|Ψ⟩──|Snap|─╭●───────────────────────────────────────╭●─╭●─╭I─╭●──────────────────────── ···
x0: ─├|Ψ⟩──|Snap|─╰X────────────────────────────────────╭●─│──╰X─├I─╰X──────────────────────── ···
y1: ─├|Ψ⟩──|Snap|─╭●──────────────────────────────╭●─╭●─│──╰X─╭●─├I─╭●──────────────────────╭● ···
x1: ─├|Ψ⟩──|Snap|─╰X───────────────────────────╭●─│──╰X─╰X────╰X─├I─╰X───────────────────╭●─│─ ···
y2: ─├|Ψ⟩──|Snap|─╭●─────────────────────╭●─╭●─│──╰X─╭●──────────├I─╭●─────────────╭●─╭●─│──╰X ···
x2: ─├|Ψ⟩──|Snap|─╰X──────────────────╭●─│──╰X─╰X────╰X──────────├I─╰X──────────╭●─│──╰X─╰X─── ···
y3: ─├|Ψ⟩──|Snap|─╭●────────────╭●─╭●─│──╰X─╭●───────────────────├I─╭●────╭●─╭●─│──╰X─╭●────── ···
x3: ─├|Ψ⟩──|Snap|─╰X─────────╭●─│──╰X─╰X────╰X───────────────────├I─╰X─╭●─│──╰X─╰X────╰X────── ···
y4: ─├|Ψ⟩──|Snap|─────────╭●─│──╰X─╭●────────────────────────────├I─╭●─│──╰X─╭●─────────────── ···
x4: ─╰|Ψ⟩──|Snap|──|Snap|─╰X─╰X────╰X────────────────────────────╰I─╰X─╰X────╰X─────────────── ···

y0: ··· ─

In [37]:
np.shape(qml.snapshots(paolo_f)(angles)["Encoded state"])
np.sum(np.abs(qml.snapshots(paolo_f)(angles)["Encoded state"])**2)
qml.snapshots(paolo_f)(angles)

{'Encoded state': array([0.70710678+0.j, 0.70710678+0.j, 0.        +0.j, ...,
        0.        +0.j, 0.        +0.j, 0.        +0.j], shape=(1024,)),
 1: np.float64(0.0),
 'execution_results': [np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.9999999999999998),
  np.float64(0.0)]}

In [ ]:
from timeit import default_timer as timer
tst = timer()
rand_ang = random_angles([4,21,3])
out = paolo_f(rand_ang)
tend = timer()
print(tend - tst)

0.01939990600112651


In [14]:
wires = [f"q{i}" for i in range(5)]
dev = qml.device('default.clifford', wires = wires)

@sum_of_expvals
@qml.qnode(dev)
def simple_f(angles, input):
    basis_encoding(input, dev.wires)
    simple_test_circuit(angles, dev.wires)
    ops = [qml.expval(qml.Z(wire)) for wire in dev.wires]
    return ops

In [15]:
simple_f(random_angles(5), 3)

np.float64(-1.0)

In [16]:
print(qml.draw(simple_f)(random_angles(5), 0))

In [36]:
analytic_NTK(simple_f, 0, 1, len(wires), 500)

np.float64(1.474)

In [37]:
4**5

1024

In [5]:
#out = simple_f(random_angles(len(wires)), 0)

print(simple_f(random_angles(len(wires)), 0))


0.0


In [36]:
type(out[0])

numpy.float64